# interactions

> Fill in a module description here

In [ ]:
#| default_exp interactions

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import dspy

In [ ]:
#| hide
import os

turbo = dspy.OpenAI(model='gpt-3.5-turbo', api_key=os.environ["OPENAI_API_KEY"])
dspy.settings.configure(lm=turbo)

In [ ]:
#| export
class BasicSeeder(dspy.Module): pass

In [ ]:
#| export
class EvolSeeder(dspy.Module): pass

In [ ]:
#| export
class GenerateQuestionOrInstruction(dspy.Signature):
    """Given a topic and a history of a conversation, generate a question or instruction for the next turn as a user."""

    topic = dspy.InputField(desc="The topic of the conversation.")
    history = dspy.InputField(desc="The history of the conversation. It can be empty.")
    question_or_instruction = dspy.OutputField(desc="The question or instruction for the next turn as a user. End with ---.")

class GenerateAnswer(dspy.Signature):
    """Given a topic and a history of a conversation, generate an answer for the next turn as an Assistant"""

    topic = dspy.InputField(desc="The topic of the conversation.")
    history = dspy.InputField(desc="The history of the conversation with the last turn being the user's question or instruction.")
    answer = dspy.OutputField(desc="The answer for the next turn as an Assistant. End with ---.")

In [ ]:
#| export
class Conversation(dspy.Module):
    def __init__(self, tools=[]):
        super().__init__()
        self.tools = tools
        self.conversation_starter = dspy.Predict(GenerateQuestionOrInstruction)
        self.answer_generator = dspy.Predict(GenerateAnswer)
        self.follow_upper = dspy.Predict(GenerateQuestionOrInstruction)
        if len(self.tools) > 0:
            self.tool_usage = dspy.ChainOfThought('topic, conversation_history, tools -> tool_usage', n = 5)

    def forward(self, topic, num_turns=1):
        initial = self.conversation_starter(topic=topic, history="Empty")
        history = [initial.question_or_instruction]
        for i in range(num_turns):
            answer = self.answer_generator(topic=topic, history="\n\n".join(history))
            history.append(answer.answer)
            if i < num_turns - 1:
                follow_up = self.follow_upper(topic=topic, history="\n\n".join(history))
                history.append(follow_up.question_or_instruction)

        return history

In [ ]:
conversation = Conversation()
num_turns = 3
history = conversation('How to make a cake', num_turns=num_turns)

assert len(history) == num_turns * 2

In [ ]:
conversation = Conversation()
num_turns = 3
history = conversation('<image_uploaded>: The image depicts - modern Chinese herbal medicine shop with a few cats sitting around, retro comic book style', num_turns=num_turns)

assert len(history) == num_turns * 2

In [ ]:
history

['What are your thoughts on the combination of modern Chinese herbal medicine and retro comic book style in this image?',
 "I find the combination of modern Chinese herbal medicine and retro comic book style in this image to be quite intriguing. It's a unique blend of traditional and contemporary elements, which adds a sense of nostalgia and whimsy to the scene. The presence of the cats also adds a touch of charm. Overall, it's a visually appealing and creative representation of the fusion between traditional practices and modern aesthetics.",
 'What other elements do you think could be added to this image to further enhance the fusion between modern Chinese herbal medicine and retro comic book style?',
 'To further enhance the fusion between modern Chinese herbal medicine and retro comic book style in this image, additional elements could be incorporated. For example, you could include traditional Chinese herbal medicine ingredients or plants in the background, such as ginseng or goji

In [ ]:
#| export
class BasicAgent(dspy.Module):
    def __init__(self):
        super().__init__()
        self.intent_generator = dspy.Predict("observation, topic -> intent")
        self.plan_generator = dspy.Predict(GenerateAnswer)
        self.action_generator = dspy.Predict(GenerateQuestionOrInstruction)

    def forward(self):
        pass

In [ ]:
#| export
class AlphaAgent(dspy.Module):
    def __init__(self):
        super().__init__()
        self.intent_generator = dspy.Predict("observation, topic -> intent")
        self.analysis_generator = dspy.Predict("observation, intent -> analysis")
        self.solution_generator = dspy.Predict("observation, intent, analysis -> solution")
        self.solution_ranker = dspy.Predict("observation, intent, solutions -> ranked_solutions")
        self.action_generator = dspy.Predict("observation, analysis, ranked_solutions, action_history -> action")

    def forward(self, observation, topic=None, action_history=None, num_solutions=None):
        pass

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()